# Locations info Arrangement

In [1]:
import json
import pandas as pd
import requests

## Hospitals

In [2]:
with open('data/fangcang-hospitals.json', 'r') as f:
    fangcang = json.loads(f.read())

In [3]:
with open('data/hospitals-wuhan.json', 'r') as f:
    hospitals = json.loads(f.read())

In [4]:
with open('data/huoshen-leishen-hospital.json', 'r') as f:
    huoshen_leishen = json.loads(f.read())

In [5]:
locations_df = pd.DataFrame()

Convert fangcang data:

In [6]:
for item in fangcang['data']:
    dfx = pd.DataFrame()
    dfx['type'] = ['Fangcang']
    dfx['title'] = [item['title']]
    dfx['address'] = [item['address']]
    dfx['lat'] = [item['location']['lat']]
    dfx['lng'] = [item['location']['lng']]
    locations_df = locations_df.append(dfx, ignore_index=True)

Convert hospitals data:

In [7]:
for item in hospitals['data']['data']:
    dfx = pd.DataFrame()
    dfx['type'] = ['Core' if item['isCore']  == 1 else 'Fever']
    dfx['title'] = [item['orgName']]
    dfx['address'] = [item['orgAddr']]
    dfx['lat'] = [item['lat']]
    dfx['lng'] = [item['lng']]
    locations_df = locations_df.append(dfx, ignore_index=True)

Convert huoshen leishen data:

In [8]:
for item in huoshen_leishen['data']:
    dfx = pd.DataFrame()
    dfx['type'] = ['HuoshenLeishen']
    dfx['title'] = [item['title']]
    dfx['address'] = [item['address']]
    dfx['lat'] = [item['location']['lat']]
    dfx['lng'] = [item['location']['lng']]
    locations_df = locations_df.append(dfx, ignore_index=True)

In [9]:
locations_df.to_csv('data/hospitals.csv', index=False)

## Storehouse

In [10]:
with open('data/store.json', 'r') as f:
    store = json.loads(f.read())

In [11]:
for item in store['data']:
    dfx = pd.DataFrame()
    dfx['type'] = ['Storehouse']
    dfx['title'] = [item['title']]
    dfx['address'] = [item['address']]
    dfx['lat'] = [item['location']['lat']]
    dfx['lng'] = [item['location']['lng']]
    locations_df = locations_df.append(dfx, ignore_index=True)

In [12]:
locations_df.to_csv('data/loacations.csv', index=False)

In [13]:
locations_df

,type,title,address,lat,lng
0,Fangcang,方舱医院(新洲区),湖北省武汉市新洲区邾城全民健身中心(永安大道东),30.837821,114.808144
1,Fangcang,方舱医院(白马驿站),湖北省武汉市洪山区欢乐大道,30.598630,114.425040
2,Fangcang,长江新城方舱医院,湖北省武汉市江岸区谌家矶特4号,30.681784,114.366320
3,Fangcang,方舱医院(洪山体育馆),湖北省武汉市武昌区体育馆路特1号,30.545917,114.334634
4,Fangcang,方舱医院(石牌岭高级职业中学),湖北省武汉市洪山区石牌岭路13号,30.524705,114.342449
...,...,...,...,...,...
103,Storehouse,武汉市江汉区应急管理局,湖北省武汉市江汉区天门墩路7号,30.605740,114.275500
104,Storehouse,武汉市东西湖区应急管理局,湖北省武汉市东西湖区啤砖路51号,30.651440,114.139590
105,Storehouse,武汉市武昌区应急管理局,湖北省武汉市武昌区中山路307号,30.553850,114.315679
106,Storehouse,武汉市硚口区应急管理局,湖北省武汉市硚口区沿河大道518号,30.582290,114.214910


## Get Distances

In [14]:
locations_df[['lat', 'lng']]

,lat,lng
0,30.837821,114.808144
1,30.598630,114.425040
2,30.681784,114.366320
3,30.545917,114.334634
4,30.524705,114.342449
...,...,...
103,30.605740,114.275500
104,30.651440,114.139590
105,30.553850,114.315679
106,30.582290,114.214910


In [15]:
map_key = 'KNHBZ-SNFCD-JCC4P-HAWVD-A2KOO-OOBHO'
url = f'http://apis.map.qq.com/ws/distance/v1/matrix?key={map_key}&mode=driving'

from_list = []
from_str = ''
for index, item in locations_df[locations_df['type'] == 'Storehouse'][['lat', 'lng']].iterrows():
    lat = item['lat']
    lng = item['lng']
    from_str += f'{lat},{lng};'
    from_list.append({'lat':lat, 'lng':lng})
from_str = from_str[:-1]

def get_to_str(type):
    to_list = []
    to_str = ''
    locations_df_temp = locations_df if type == 'All' else locations_df[locations_df['type'] == type]
    print(type, len(locations_df_temp))
    for index, item in locations_df_temp[['lat', 'lng']].iterrows():
        lat = item['lat']
        lng = item['lng']
        to_str += f'{lat},{lng};'
        to_list.append({'lat':lat, 'lng':lng})

    to_str = to_str[:-1]
    print(type, '\n', to_list)
    return to_str

to_fangcang_str = get_to_str('Fangcang')
to_huolei_str = get_to_str('HuoshenLeishen')
to_core_str = get_to_str('Core')
to_fever_str = get_to_str('Fever')

url_fangcang = f'{url}&from={from_str}&to={to_fangcang_str}'
url_huolei = f'{url}&from={from_str}&to={to_huolei_str}'
url_core = f'{url}&from={from_str}&to={to_core_str}'
url_fever = f'{url}&from={from_str}&to={to_fever_str}'


Fangcang 20
Fangcang 
 [{'lat': 30.837821, 'lng': 114.808144}, {'lat': 30.59863, 'lng': 114.42504}, {'lat': 30.681784, 'lng': 114.36632}, {'lat': 30.545917, 'lng': 114.334634}, {'lat': 30.524705, 'lng': 114.342449}, {'lat': 30.621089, 'lng': 114.228881}, {'lat': 30.577, 'lng': 114.260066}, {'lat': 30.500686, 'lng': 114.171899}, {'lat': 30.612412, 'lng': 114.388901}, {'lat': 30.39347, 'lng': 114.426479}, {'lat': 30.90544, 'lng': 114.360695}, {'lat': 30.641, 'lng': 114.284711}, {'lat': 30.502381, 'lng': 114.059876}, {'lat': 30.507009, 'lng': 114.242098}, {'lat': 30.580293, 'lng': 114.273331}, {'lat': 30.699498, 'lng': 114.235175}, {'lat': 30.452971, 'lng': 114.163264}, {'lat': 30.490452, 'lng': 114.509388}, {'lat': 30.357564, 'lng': 114.348006}, {'lat': 30.569881, 'lng': 114.219133}]
HuoshenLeishen 2
HuoshenLeishen 
 [{'lat': 30.527887, 'lng': 114.087392}, {'lat': 30.430179, 'lng': 114.291359}]
Core 38
Core 
 [{'lat': 30.57702, 'lng': 114.02759}, {'lat': 30.561282, 'lng': 114.04347}, {'l

In [16]:
def translate_map(type, res_map, fixed_row = -1):
    locations_df_rtemp = locations_df if type == 'All' else locations_df[locations_df['type'] == 'Storehouse']
    locations_df_ctemp = locations_df if type == 'All' else locations_df[locations_df['type'] == type]
    rows = list(locations_df_rtemp['title'])
    cols = list(locations_df_ctemp['title'])

    df_distance = pd.DataFrame(columns=cols)
    df_duration = pd.DataFrame(columns=cols)
    for index, row in enumerate(res_map['rows']):
        dfx = pd.DataFrame()
        dfy = pd.DataFrame()
        if fixed_row >= 0:
            dfx['storehouse'] = [rows[fixed_row]]
            dfy['storehouse'] = [rows[fixed_row]]
        else:
            dfx['storehouse'] = [rows[index]]
            dfy['storehouse'] = [rows[index]]
        for index_y, col in enumerate(row['elements']):
            dfx[cols[index_y]] = [col['distance']]
            dfy[cols[index_y]] = [col['duration']]
        df_distance = df_distance.append(dfx)
        df_duration = df_duration.append(dfy)
    
    return df_distance, df_duration

def get_distance_info(type):
    if type == 'Fangcang':
        url = url_fangcang
    elif type == 'HuoshenLeishen':
        url = url_huolei
    elif type == 'Core':
        url = url_core
    elif type == 'Fever':
        url = url_fever
    
    res = requests.get(url)
    print(url)
    res = res.json()['result']

    return translate_map(type, res)

def save_distanceinfo(type):
    dis_df, dua_df = get_distance_info(type)
    dis_df.to_csv(f'data/{type}_distance.csv', index=False)
    dua_df.to_csv(f'data/{type}_duration.csv', index=False)

In [17]:
for type in ['Fangcang', 'HuoshenLeishen', 'Core', 'Fever']:
    save_distanceinfo(type)

http://apis.map.qq.com/ws/distance/v1/matrix?key=KNHBZ-SNFCD-JCC4P-HAWVD-A2KOO-OOBHO&mode=driving&from=30.589509,114.300335;30.506843,114.241888;30.81081,114.226873;30.634988,114.292486;30.55423,114.2185;30.841568,114.807981;30.57813,114.03364;30.882697,114.377555;30.34257,114.31341;30.50054,114.34295;30.63116,114.39569;30.60574,114.2755;30.65144,114.13959;30.55385,114.315679;30.58229,114.21491;30.50574,114.167526&to=30.837821,114.808144;30.59863,114.42504;30.681784,114.36632;30.545917,114.334634;30.524705,114.342449;30.621089,114.228881;30.577,114.260066;30.500686,114.171899;30.612412,114.388901;30.39347,114.426479;30.90544,114.360695;30.641,114.284711;30.502381,114.059876;30.507009,114.242098;30.580293,114.273331;30.699498,114.235175;30.452971,114.163264;30.490452,114.509388;30.357564,114.348006;30.569881,114.219133
http://apis.map.qq.com/ws/distance/v1/matrix?key=KNHBZ-SNFCD-JCC4P-HAWVD-A2KOO-OOBHO&mode=driving&from=30.589509,114.300335;30.506843,114.241888;30.81081,114.226873;30.63

In [18]:
from_list

[{'lat': 30.589509, 'lng': 114.300335},
 {'lat': 30.506843, 'lng': 114.241888},
 {'lat': 30.81081, 'lng': 114.226873},
 {'lat': 30.634988, 'lng': 114.292486},
 {'lat': 30.55423, 'lng': 114.2185},
 {'lat': 30.841568, 'lng': 114.807981},
 {'lat': 30.57813, 'lng': 114.03364},
 {'lat': 30.882697, 'lng': 114.377555},
 {'lat': 30.34257, 'lng': 114.31341},
 {'lat': 30.50054, 'lng': 114.34295},
 {'lat': 30.63116, 'lng': 114.39569},
 {'lat': 30.60574, 'lng': 114.2755},
 {'lat': 30.65144, 'lng': 114.13959},
 {'lat': 30.55385, 'lng': 114.315679},
 {'lat': 30.58229, 'lng': 114.21491},
 {'lat': 30.50574, 'lng': 114.167526}]

In [19]:
to_str = get_to_str('All')
dis_df = pd.DataFrame()
dua_df = pd.DataFrame()

for index, item in locations_df[['lat', 'lng']].iterrows():
    lat = item['lat']
    lng = item['lng']
    from_str = f'{lat},{lng}'
    url_temp = f'{url}&from={from_str}&to={to_str}'
    res = requests.get(url_temp)
    res = res.json()['result']

    dis_df_x, dua_df_x = translate_map('All', res, fixed_row = index)
    dis_df = dis_df.append(dis_df_x, ignore_index=True)
    dua_df = dua_df.append(dua_df_x, ignore_index=True)

All 108
All 
 [{'lat': 30.837821, 'lng': 114.808144}, {'lat': 30.59863, 'lng': 114.42504}, {'lat': 30.681784, 'lng': 114.36632}, {'lat': 30.545917, 'lng': 114.334634}, {'lat': 30.524705, 'lng': 114.342449}, {'lat': 30.621089, 'lng': 114.228881}, {'lat': 30.577, 'lng': 114.260066}, {'lat': 30.500686, 'lng': 114.171899}, {'lat': 30.612412, 'lng': 114.388901}, {'lat': 30.39347, 'lng': 114.426479}, {'lat': 30.90544, 'lng': 114.360695}, {'lat': 30.641, 'lng': 114.284711}, {'lat': 30.502381, 'lng': 114.059876}, {'lat': 30.507009, 'lng': 114.242098}, {'lat': 30.580293, 'lng': 114.273331}, {'lat': 30.699498, 'lng': 114.235175}, {'lat': 30.452971, 'lng': 114.163264}, {'lat': 30.490452, 'lng': 114.509388}, {'lat': 30.357564, 'lng': 114.348006}, {'lat': 30.569881, 'lng': 114.219133}, {'lat': 30.57702, 'lng': 114.02759}, {'lat': 30.561282, 'lng': 114.04347}, {'lat': 30.648869, 'lng': 114.132874}, {'lat': 30.548545, 'lng': 114.112783}, {'lat': 30.314335, 'lng': 114.07301}, {'lat': 30.3013, 'lng': 1

In [20]:
dis_df

,方舱医院(新洲区),方舱医院(白马驿站),长江新城方舱医院,方舱医院(洪山体育馆),方舱医院(石牌岭高级职业中学),方舱医院(江汉经济开发区),方舱医院(武汉体育馆),方舱医院(武汉体育中心),方舱医院(武钢体育馆),方舱医院(中国光谷日海),...,黄陂区应急管理局,武汉市江夏区应急管理局,洪山区应急管理局,武汉市青山区应急管理局,武汉市江汉区应急管理局,武汉市东西湖区应急管理局,武汉市武昌区应急管理局,武汉市硚口区应急管理局,武汉市经济技术开发区(汉南区)应急管理局,storehouse
0,0,62120,54991,72216,75179,71088,76707,86193,66524,84932,...,55433,101073,76953,63009,71856,85432,75128,76879,86002,方舱医院(新洲区)
1,61319,0,15424,15357,18320,27788,24353,39727,9665,29364,...,48868,38700,20094,7913,23654,38390,18270,30413,39537,方舱医院(白马驿站)
2,55508,15100,0,20522,23266,19019,17851,30958,15564,42088,...,33128,44480,25040,13157,14884,29621,19871,21644,30768,长江新城方舱医院
3,73265,12299,20955,0,3839,19892,10906,23915,11826,26799,...,45996,26637,7197,15335,11914,30554,2860,19575,23725,方舱医院(洪山体育馆)
4,75306,14340,22903,4333,0,22914,12355,22624,15093,24736,...,49653,24574,5134,18029,14339,33941,5061,19455,22434,方舱医院(石牌岭高级职业中学)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103,72836,21626,16243,10787,14423,7103,5356,19749,17387,42474,...,37732,52007,17363,17368,0,17764,9324,8423,19559,武汉市江汉区应急管理局
104,86350,36188,32296,28922,34816,12178,19680,21674,31949,53442,...,39485,47167,35978,31930,19172,0,27459,14938,21484,武汉市东西湖区应急管理局
105,75473,14506,20425,2894,5338,19700,10714,23723,12099,28933,...,44243,41132,9013,13333,9853,30362,0,19383,23533,武汉市武昌区应急管理局
106,78181,27317,21933,16626,20128,9190,4979,12241,23078,40633,...,42853,49320,21290,23059,10264,14168,15162,0,12051,武汉市硚口区应急管理局


In [21]:
dis_df.to_csv('data/distances.csv', index=False)